# Dependencies and SetUp

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

# Load csv exported from WeatherPy to a DataFrame

In [2]:
csv_file = 'output_data/cities.csv'
cities_df = pd.read_csv(csv_file)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kayerkan,69.3497,87.7602,35.13,97,100,12.46,RU,1632081430
1,Mataura,-46.1927,168.8643,39.47,96,32,3.87,NZ,1632081431
2,Koungou,-12.7336,45.2042,76.71,78,90,14.97,YT,1632081431
3,Severo-Kuril'sk,50.6789,156.1250,50.45,97,100,12.35,RU,1632081158
4,Nyurba,63.2842,118.3319,23.85,91,10,2.26,RU,1632081432


# Humidity Heatmap

* use Lat and Lng as locations and Humidity as weight
* Add Heatmap Layer to Map

In [3]:
locations = cities_df[['Lat','Lng']]
weight = cities_df['Humidity']
max_intensity = weight.max()

# Plot Heatmap
fig = gmaps.figure(center=(46.0,-5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights= weight, dissipating=False, max_intensity=max_intensity, point_radius=1)

# Add Layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

#  Create new DataFrame fitting weather criteria

* Narrow down the DataFrame to find your ideal weather condition
* Drop any rows that don't contain all three conditions.

In [4]:
# Filter the data to fit the ideal weather condition
ideal_weather = cities_df.loc[(cities_df['Max Temp'] >= 75) & (cities_df['Max Temp'] < 90) & \
                              (cities_df['Wind Speed'] < 11) & (cities_df['Cloudiness'] < 20) & \
                              (cities_df['Humidity'] < 90) & (cities_df['Humidity'] >= 40)].dropna()

In [5]:
# Print new total count of cities
city_number = ideal_weather['City'].count()
city_number_ori = cities_df['City'].count()

print(f'After cleaning the data, there are a new total of {city_number} out of {city_number_ori} cities that fit within the ideal weather')

After cleaning the data, there are a new total of 25 out of 585 cities that fit within the ideal weather


# Hotel Map
* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create hotel_df
hotel_df = ideal_weather.loc[:,["City",'Country','Lat','Lng']]

# Add a new column Hotel Name
hotel_df['Hotel Name'] = ""

# Display hotel_df
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
10,Westport,US,41.1415,-73.3579,
11,Caravelas,BR,-17.7125,-39.2481,
12,Albany,US,42.6001,-73.9662,
22,Jamestown,US,42.0970,-79.2353,
49,Māngrol,IN,21.1167,70.1167,


In [7]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {'type':'hotel',
         'keyword':'hotel',
         'radius': 5000,
         'key': gkey}

In [8]:

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        
    print("------------")

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 10: Westport.
Closest hotel in Westport is Norwalk Inn.
------------
Retrieving Results for Index 11: Caravelas.
Closest hotel in Caravelas is Pousada Liberdade.
------------
Retrieving Results for Index 12: Albany.
Missing field/result... skipping.
------------
Retrieving Results for Index 22: Jamestown.
Closest hotel in Jamestown is Chautauqua Harbor Hotel.
------------
Retrieving Results for Index 49: Māngrol.
Missing field/result... skipping.
------------
Retrieving Results for Index 57: Kavieng.
Closest hotel in Kavieng is Nusa Island Retreat.
------------
Retrieving Results for Index 67: Goderich.
Closest hotel in Goderich is Comfort Inn & Suites.
------------
Retrieving Results for Index 69: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
------------
Retrieving Results for Index 73: Fukuechō.
Closest hotel in Fukuechō is SERENDIP HOTEL GOTO.
------------
Retrieving Results for Index 132: Baena.
Closest hotel in Baena is La Casa Grande.
------

In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
10,Westport,US,41.1415,-73.3579,Norwalk Inn
11,Caravelas,BR,-17.7125,-39.2481,Pousada Liberdade
12,Albany,US,42.6001,-73.9662,NaN
22,Jamestown,US,42.0970,-79.2353,Chautauqua Harbor Hotel
49,Māngrol,IN,21.1167,70.1167,NaN
57,Kavieng,PG,-2.5744,150.7967,Nusa Island Retreat
67,Goderich,CA,43.7501,-81.7165,Comfort Inn & Suites
69,Kiryat Gat,IL,31.6100,34.7642,Desert Gat
73,Fukuechō,JP,32.6881,128.8419,SERENDIP HOTEL GOTO
132,Baena,ES,37.6167,-4.3225,La Casa Grande


In [14]:
# Reset index of hotel_df and drop rows with NaN values
hotel_df = hotel_df.reset_index(drop=True)
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'],axis=0)
clean_hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Westport,US,41.1415,-73.3579,Norwalk Inn
1,Caravelas,BR,-17.7125,-39.2481,Pousada Liberdade
3,Jamestown,US,42.0970,-79.2353,Chautauqua Harbor Hotel
5,Kavieng,PG,-2.5744,150.7967,Nusa Island Retreat
6,Goderich,CA,43.7501,-81.7165,Comfort Inn & Suites
7,Kiryat Gat,IL,31.6100,34.7642,Desert Gat
8,Fukuechō,JP,32.6881,128.8419,SERENDIP HOTEL GOTO
9,Baena,ES,37.6167,-4.3225,La Casa Grande
10,Darnah,LY,32.7670,22.6367,فندق النعيم
11,Roanoke Rapids,US,36.4615,-77.6542,Hampton Inn Roanoke Rapids


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))